In [1]:
pip install keras==2.6.*

Note: you may need to restart the kernel to use updated packages.


In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
seed(1)
import pandas as pd
import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LeakyReLU

2021-12-20 15:57:19.862444: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 15:57:19.862467: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
df= pd.read_csv('train.csv')
X = df.drop(columns = 'critical_temp')
y = df['critical_temp']
y = np.array(y)

In [10]:
df

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21259,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21260,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21261,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


In [15]:
df.shape

(21263, 82)

In [5]:
#Filter the data
cor_matrix = df.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]#IF THE CORRELATION IS HIGHER THAN 0.9 WE CAN DROP THE COLUMNS
df_copy = df.copy(deep=True)
df_copy.drop(columns=to_drop,inplace=True)
X = df_copy.drop(columns = ['critical_temp'])
y = df.critical_temp
y = np.array(y)
X = np.array(X)
X_train,X_test,y_train,y_test = train_test_split(X,y)

/tmp/ipykernel_38958/1746069364.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))


## Scaling data


In [6]:
y_train=np.reshape(y_train, (-1,1))
y_test=np.reshape(y_test, (-1,1))
scaler_x = MinMaxScaler()
print(scaler_x.fit(X_train))
xtrain_scale=scaler_x.transform(X_train)
print(scaler_x.fit(X_test))
xval_scale=scaler_x.transform(X_test)

MinMaxScaler()
MinMaxScaler()


In [7]:
xtrain_scale.shape

(15947, 43)

In [8]:
model = Sequential()
model.add(Dense(43, input_dim = 43, kernel_initializer='normal', activation='relu'))
model.add(Dense(3000,activation='relu',kernel_initializer='normal'))
model.add(Dense(2500,activation='relu',kernel_initializer='normal'))
model.add(Dense(2000,activation='relu', kernel_initializer='normal'))
model.add(Dense(1500,activation='relu',kernel_initializer='normal'))
model.add(Dense(1000,activation='relu',kernel_initializer='normal'))
model.add(Dense(500,activation='relu', kernel_initializer='normal'))
model.add(Dense(300,activation='relu',kernel_initializer='normal'))
model.add(Dense(150,activation='relu',kernel_initializer='normal'))
model.add(Dense(75,activation='relu', kernel_initializer='normal'))
model.add(Dense(1,kernel_initializer='normal'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 43)                1892      
_________________________________________________________________
dense_1 (Dense)              (None, 3000)              132000    
_________________________________________________________________
dense_2 (Dense)              (None, 2500)              7502500   
_________________________________________________________________
dense_3 (Dense)              (None, 2000)              5002000   
_________________________________________________________________
dense_4 (Dense)              (None, 1500)              3001500   
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1501000   
_________________________________________________________________
dense_6 (Dense)              (None, 500)               5

2021-12-20 15:59:06.334652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-20 15:59:06.334689: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-20 15:59:06.334722: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mobile): /proc/driver/nvidia/version does not exist
2021-12-20 15:59:06.335503: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
model.compile(loss='mse', optimizer='adam', metrics=[tensorflow.keras.metrics.RootMeanSquaredError()])
training=model.fit(xtrain_scale, y_train, epochs=30, batch_size=100, verbose=1, validation_split=0.2)

Epoch 1/30
128/128 [==============================] - 11s 85ms/step - loss: 315.4663 - root_mean_squared_error: 20.6856 - val_loss: 211.9978 - val_root_mean_squared_error: 17.4495
Epoch 2/30
128/128 [==============================] - 11s 83ms/step - loss: 244.2948 - root_mean_squared_error: 16.7123 - val_loss: 206.9920 - val_root_mean_squared_error: 16.4046
Epoch 3/30
128/128 [==============================] - 11s 84ms/step - loss: 230.2813 - root_mean_squared_error: 16.1392 - val_loss: 217.4611 - val_root_mean_squared_error: 15.9559
Epoch 4/30
128/128 [==============================] - 11s 83ms/step - loss: 237.6722 - root_mean_squared_error: 15.8448 - val_loss: 264.1480 - val_root_mean_squared_error: 15.8205
Epoch 5/30
128/128 [==============================] - 11s 84ms/step - loss: 245.7680 - root_mean_squared_error: 15.8293 - val_loss: 198.2969 - val_root_mean_squared_error: 15.7734
Epoch 6/30
128/128 [==============================] - 11s 84ms/step - loss: 225.4070 - root_mean_squ